# Geospatial Data Analysis Lab: Steel Plants Dataset


**(15/10/2025) Learning Objectives:**
- Perform exploratory data analysis (EDA) on geospatial datasets
- Visualize geospatial data using interactive maps with Plotly
- Merge environmental data with asset locations
- Aggregate data at the company level
- Integrate geospatial visualizations into a Streamlit dashboard

---


## Part 1: Setup and Data Loading

Import the necessary libraries and load the steel plants dataset.


In [2]:
# Import required libraries
# - pandas for data manipulation
# - numpy for numerical operations
# - plotly.express and plotly.graph_objects for interactive visualizations
# - Any other libraries you might need

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt



In [3]:
# Load the steel plants dataset from Excel file
# The dataset has multiple sheets - we need to merge "Plant data" and "Plant capacities and status"
excel_file = 'Dataset/Plant-level-data-Global-Iron-and-Steel-Tracker-September-2025-V1.xlsx'

# Load both sheets
plant_data = pd.read_excel(excel_file, sheet_name='Plant data')
capacity_data = pd.read_excel(excel_file, sheet_name='Plant capacities and status')

print(f"Plant data shape: {plant_data.shape}")
print(f"Capacity data shape: {capacity_data.shape}")

# Merge on Plant ID - using left merge to keep all plants from plant_data
# Some plants may have multiple capacity records, so we'll need to handle duplicates
df = pd.merge(plant_data, capacity_data, on='Plant ID', how='left', suffixes=('', '_capacity'))

print(f"\nMerged data shape (before filtering): {df.shape}")

# Filter to keep only operating plants
print(f"\nStatus distribution before filtering:")
print(df['Status'].value_counts())

df = df[df['Status'] == 'operating'].copy()

print(f"\nAfter filtering for 'operating' status:")
print(f"Final dataset shape: {df.shape}")
print(f"Number of operating plants: {len(df)}")

# Display first few rows
df.head()

Plant data shape: (1209, 44)
Capacity data shape: (1744, 15)

Merged data shape (before filtering): (1744, 58)

Status distribution before filtering:
Status
operating                    868
announced                    285
retired                      181
construction                 151
operating pre-retirement     125
mothballed                    73
cancelled                     59
mothballed pre-retirement      2
Name: count, dtype: int64

After filtering for 'operating' status:
Final dataset shape: (868, 58)
Number of operating plants: 868


,Plant ID,Plant name (English),Plant name (other language),Other plant names (English),Other plant names (other language),Owner,Owner (other language),Owner GEM ID,Owner PermID,SOE Status,...,Start date_capacity,Nominal crude steel capacity (ttpa),Nominal BOF steel capacity (ttpa),Nominal EAF steel capacity (ttpa),Nominal OHF steel capacity (ttpa),Other/unspecified steel capacity (ttpa),Nominal iron capacity (ttpa),Nominal BF capacity (ttpa),Nominal DRI capacity (ttpa),Other/unspecified iron capacity (ttpa)
1,P100000120439,Algerian Qatari Steel Jijel plant,الجزائرية القطرية للصلب,AQS,NaN,Algerian Qatari Steel,NaN,E100001000957,5076384326,Partial,...,2017,2200,NaN,2200,NaN,NaN,2500,NaN,2500,NaN
4,P100000121198,Ozmert Algeria steel plant,NaN,NaN,NaN,Ozmert Algeria SARL,NaN,E100001012196,unknown,NaN,...,unknown,800,NaN,800,NaN,NaN,500,NaN,500,NaN
5,P100000120440,Sider El Hadjar Annaba steel plant,مركب الحجار للحديد والصلب,"ArcelorMittal Annaba (predecessor), El Hadjar ...",NaN,Groupe Industriel Sider SpA,NaN,E100001000960,5000941519,Full,...,1969,2150,350.0,1800,NaN,NaN,1500,1500,NaN,NaN
7,P100000120441,Tosyali Algerie Oran steel plant,شركة توسيالي الجزائرية التركية للحديد والصلب,NaN,Tosyali Algérie,Tosyali Ironsteel Industry Algerie SpA,NaN,E100000131071,5074196906,NaN,...,2013,6200,NaN,6200,NaN,NaN,5000,NaN,5000,NaN
8,P100000120005,Aceria Angola Bengo steel plant,NaN,ADA Steel,NaN,Ada - Aceria De Angola SA,NaN,E100000131097,unknown,NaN,...,2015,500,NaN,NaN,NaN,500,NaN,NaN,NaN,NaN


---
## Part 2: Exploratory Data Analysis (15 minutes)

Answer the following questions through your analysis:


### Question 1: Data Overview
**Task:** Display basic information about the dataset.
- How many steel plants are in the dataset?
- What are the column names and data types?
- Are there any missing values?


In [4]:
# Question 2: Statistical Summary - Average Plant Capacity
# This cell analyzes the capacity distribution of steel plants

capacity_column = 'Nominal crude steel capacity (ttpa)'

# Convert the selected capacity column to numeric
# This ensures we can perform mathematical operations on capacity values
if capacity_column in df.columns:
    df[capacity_column] = pd.to_numeric(df[capacity_column], errors='coerce')

print("\nCapacity Statistics (in thousand tonnes per annum - ttpa):\n")

# Calculate and display comprehensive capacity statistics
# Only proceed if we have valid capacity data
if capacity_column in df.columns:
    non_null_count = df[capacity_column].notna().sum()
    if non_null_count > 0:
        print(f"{capacity_column}:")
        print(f"  Plants with data: {non_null_count}")
        print(f"  Average: {df[capacity_column].mean():,.2f} ttpa")
        print(f"  Median: {df[capacity_column].median():,.2f} ttpa")
        print(f"  Min: {df[capacity_column].min():,.2f} ttpa")
        print(f"  Max: {df[capacity_column].max():,.2f} ttpa")
        print(f"  Total: {df[capacity_column].sum():,.2f} ttpa")
        print()


# 1. Histogram of crude steel capacity distribution
# Visualize the distribution of plant capacities to understand industry structure
fig1 = px.histogram(
    df.dropna(subset=['Nominal crude steel capacity (ttpa)']),
    x='Nominal crude steel capacity (ttpa)',
    nbins=50,  # Use 50 bins for detailed distribution view
    title='Distribution of Crude Steel Capacity (Operating Plants)',
    labels={'Nominal crude steel capacity (ttpa)': 'Capacity (ttpa)'},
    color_discrete_sequence=['#1f77b4']  # Consistent blue color scheme
)
fig1.update_layout(
    xaxis_title='Capacity (thousand tonnes per annum)',
    yaxis_title='Number of Plants',
    showlegend=False,
    height=400
)
fig1.show()

# 2. Top 20 plants by crude steel capacity
# Identify the largest steel plants globally for capacity analysis
top_plants = df.nlargest(20, 'Nominal crude steel capacity (ttpa)')[
    ['Plant name (English)', 'Country/Area', 'Nominal crude steel capacity (ttpa)', 'Owner']
].copy()

fig2 = px.bar(
    top_plants,
    y='Plant name (English)',
    x='Nominal crude steel capacity (ttpa)',
    title='Top 20 Operating Plants by Crude Steel Capacity',
    labels={'Nominal crude steel capacity (ttpa)': 'Capacity (ttpa)', 'Plant name (English)': 'Plant'},
    hover_data=['Country/Area', 'Owner'],  # Show additional context on hover
    color='Nominal crude steel capacity (ttpa)',
    color_continuous_scale='Viridis',  # Color gradient for visual appeal
    orientation='h'  # Horizontal bars for better readability
)
fig2.update_layout(
    xaxis_title='Capacity (ttpa)',
    yaxis_title='',
    height=600,  # Taller height to accommodate 20 plant names
    showlegend=False
)
fig2.show()



Capacity Statistics (in thousand tonnes per annum - ttpa):

Nominal crude steel capacity (ttpa):
  Plants with data: 815
  Average: 2,459.65 ttpa
  Median: 1,350.00 ttpa
  Min: 13.00 ttpa
  Max: 22,999.00 ttpa
  Total: 2,004,616.52 ttpa



ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
# Question 2: Statistical Summary - Latitude and Longitude Range
# This cell analyzes the geographic distribution of steel plants

# Split coordinates column into latitude and longitude
# The Coordinates column contains comma-separated lat,lon values that need to be parsed
df[['latitude', 'longitude']] = df['Coordinates'].str.split(',', expand=True)
# Convert string coordinates to numeric values for mathematical operations
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

# Display comprehensive geographic range statistics
print(f"Latitude range:")
print(f"  Min: {df['latitude'].min():.4f}°")
print(f"  Max: {df['latitude'].max():.4f}°")
print(f"  Mean: {df['latitude'].mean():.4f}°")
print(f"\nLongitude range:")
print(f"  Min: {df['longitude'].min():.4f}°")
print(f"  Max: {df['longitude'].max():.4f}°")
print(f"  Mean: {df['longitude'].mean():.4f}°")


In [ ]:
# Question 2: Statistical Summary - Plant Age Distribution
# This cell analyzes the age distribution of steel plants

# Convert plant age to numeric for statistical analysis
df['Plant age (years)'] = pd.to_numeric(df['Plant age (years)'], errors='coerce')

# Display age statistics
plants_with_age = df['Plant age (years)'].notna().sum()
print(f"\nPlants with age data: {plants_with_age} out of {len(df)}")
print(f"\nAge Statistics:")
print(f"  Average age: {df['Plant age (years)'].mean():.1f} years")
print(f"  Median age: {df['Plant age (years)'].median():.1f} years")
print(f"  Min age: {df['Plant age (years)'].min():.1f} years")
print(f"  Max age: {df['Plant age (years)'].max():.1f} years")

# Create histogram of plant ages
fig = px.histogram(
    df.dropna(subset=['Plant age (years)']),
    x='Plant age (years)',
    nbins=30,  # Use 30 bins for age distribution
    title='Distribution of Steel Plant Ages',
    labels={'Plant age (years)': 'Age (years)'},
    color_discrete_sequence=['#2ca02c']  # Green color for age data
)
fig.update_layout(
    xaxis_title='Plant Age (years)',
    yaxis_title='Number of Plants',
    showlegend=False,
    height=400
)
fig.show()


In [ ]:
# Display dataset shape
print("Number of steel plants:", df.shape[0])
print("Dataset shape (rows, columns):", df.shape)

Number of steel plants: 868
Dataset shape (rows, columns): (868, 58)


In [ ]:
# Display column information and data types
print("\nColumn names and data types:")
print(df.info())


Column names and data types:
<class 'pandas.core.frame.DataFrame'>
Index: 868 entries, 1 to 1742
Data columns (total 58 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Plant ID                                 868 non-null    object 
 1   Plant name (English)                     868 non-null    object 
 2   Plant name (other language)              533 non-null    object 
 3   Other plant names (English)              549 non-null    object 
 4   Other plant names (other language)       222 non-null    object 
 5   Owner                                    868 non-null    object 
 6   Owner (other language)                   411 non-null    object 
 7   Owner GEM ID                             868 non-null    object 
 8   Owner PermID                             868 non-null    object 
 9   SOE Status                               148 non-null    object 
 10  Parent                  

In [ ]:
# Check for missing values
print("\nMissing values per column:")
print(df.isnull().sum())
total_missing = df.isnull().sum().sum()
print("\nTotal number of missing values in dataset:", total_missing)


Missing values per column:
Plant ID                                     0
Plant name (English)                         0
Plant name (other language)                335
Other plant names (English)                319
Other plant names (other language)         646
Owner                                        0
Owner (other language)                     457
Owner GEM ID                                 0
Owner PermID                                 0
SOE Status                                 720
Parent                                       0
Parent GEM ID                                0
Parent PermID                                0
Location address                             0
Municipality                                 0
Subnational unit (province/state)            0
Country/Area                                 0
Region                                       0
Other language location address            545
Coordinates                                  0
Coordinate accuracy             

### Question 2: Statistical Summary
**Task:** Generate descriptive statistics for numerical columns.
- What is the average plant capacity?
- What is the range of latitudes and longitudes?
- What is the distribution of plant ages?


What is the average plant capacity?

In [ ]:
# Define the column name containing plant production capacity data (in thousand tonnes per annum)
capacity_column = 'Nominal crude steel capacity (ttpa)'

# Ensure the capacity data is numeric for statistical and plotting purposes.
# Non-numeric entries (e.g., text, missing values) are coerced to NaN to avoid calculation errors.
if capacity_column in df.columns:
    df[capacity_column] = pd.to_numeric(df[capacity_column], errors='coerce')

print("\nCapacity Statistics (in thousand tonnes per annum - ttpa):\n")

# Check that the capacity column exists before proceeding
if capacity_column in df.columns:
    # Count how many records contain valid (non-null) capacity data
    non_null_count = df[capacity_column].notna().sum()
    
    # Proceed only if there is at least one valid value
    if non_null_count > 0:
        print(f"{capacity_column}:")
        print(f"  Plants with data: {non_null_count}")  # Number of plants with valid capacity info
        print(f"  Average: {df[capacity_column].mean():,.2f} ttpa")  # Mean capacity
        print(f"  Median: {df[capacity_column].median():,.2f} ttpa")  # Median capacity
        print(f"  Min: {df[capacity_column].min():,.2f} ttpa")  # Smallest plant capacity
        print(f"  Max: {df[capacity_column].max():,.2f} ttpa")  # Largest plant capacity
        print(f"  Total: {df[capacity_column].sum():,.2f} ttpa")  # Aggregate capacity across all plants
        print()

# --- Visualization 1: Capacity distribution histogram ---
# This plot shows how crude steel production capacities are distributed across all operating plants.
# It helps identify whether most plants are small, medium, or large-scale producers.
fig1 = px.histogram(
    df.dropna(subset=['Nominal crude steel capacity (ttpa)']),  # Remove missing capacity values
    x='Nominal crude steel capacity (ttpa)',
    nbins=50,  # Number of histogram bins for granularity
    title='Distribution of Crude Steel Capacity (Operating Plants)',
    labels={'Nominal crude steel capacity (ttpa)': 'Capacity (ttpa)'},
    color_discrete_sequence=['#1f77b4']  # Use a single color tone for clarity
)
fig1.update_layout(
    xaxis_title='Capacity (thousand tonnes per annum)',  # X-axis label
    yaxis_title='Number of Plants',  # Y-axis label
    showlegend=False,
    height=400
)
fig1.show()

# --- Visualization 2: Top 20 plants by capacity ---
# Identify and visualize the largest operating plants globally or regionally.
# This provides insight into concentration and dominance within the steel industry.
top_plants = df.nlargest(20, 'Nominal crude steel capacity (ttpa)')[
    ['Plant name (English)', 'Country/Area', 'Nominal crude steel capacity (ttpa)', 'Owner']
].copy()

# Horizontal bar chart of the 20 highest-capacity plants.
# Each bar represents a single plant, with hover info showing its country and owner.
fig2 = px.bar(
    top_plants,
    y='Plant name (English)',
    x='Nominal crude steel capacity (ttpa)',
    title='Top 20 Operating Plants by Crude Steel Capacity',
    labels={
        'Nominal crude steel capacity (ttpa)': 'Capacity (ttpa)',
        'Plant name (English)': 'Plant'
    },
    hover_data=['Country/Area', 'Owner'],
    color='Nominal crude steel capacity (ttpa)',
    color_continuous_scale='Viridis',  # Gradient highlights magnitude differences
    orientation='h'  # Horizontal bars for better readability with long plant names
)
fig2.update_layout(
    xaxis_title='Capacity (ttpa)',
    yaxis_title='',
    height=600,
    showlegend=False
)
fig2.show()


NameError: name 'df' is not defined

What is the range of latitudes and longitudes?

In [ ]:
# Split coordinates column into latitude and longitude
df[['latitude', 'longitude']] = df['Coordinates'].str.split(',', expand=True)
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

print(f"Latitude range:")
print(f"  Min: {df['latitude'].min():.4f}°")
print(f"  Max: {df['latitude'].max():.4f}°")
print(f"  Mean: {df['latitude'].mean():.4f}°")
print(f"\nLongitude range:")
print(f"  Min: {df['longitude'].min():.4f}°")
print(f"  Max: {df['longitude'].max():.4f}°")
print(f"  Mean: {df['longitude'].mean():.4f}°")


Latitude range:
  Min: -37.8314°
  Max: 66.3115°
  Mean: 30.5709°

Longitude range:
  Min: -123.1636°
  Max: 174.7281°
  Mean: 59.6227°


In [ ]:
# Convert the 'Plant age (years)' column to numeric values.
# Invalid entries (e.g., text, missing) are converted to NaN to enable statistical analysis.
df['Plant age (years)'] = pd.to_numeric(df['Plant age (years)'], errors='coerce')

# Display basic information about how many plants have valid age data.
print(f"\nPlants with age data: {df['Plant age (years)'].notna().sum()} out of {len(df)}")

# Summary statistics describing the age distribution of plants.
print(f"\nAge Statistics:")
print(f"  Mean: {df['Plant age (years)'].mean():.2f} years")
print(f"  Median: {df['Plant age (years)'].median():.2f} years")
print(f"  Min: {df['Plant age (years)'].min():.0f} years")
print(f"  Max: {df['Plant age (years)'].max():.0f} years")
print(f"  Std Dev: {df['Plant age (years)'].std():.2f} years")

# Categorize plants into defined age ranges for easier interpretation.
# This helps reveal the distribution of younger vs. older plants.
print(f"\nAge Distribution by Category:")
age_bins = [0, 10, 20, 30, 40, 50, 100, 300]
age_labels = ['0-10 years', '11-20 years', '21-30 years', '31-40 years', '41-50 years', '51-100 years', '100+ years']
df['age_category'] = pd.cut(df['Plant age (years)'], bins=age_bins, labels=age_labels, right=True)
print(df['age_category'].value_counts().sort_index())

# Visualize the count of plants within each age category using a bar chart.
# This shows how plant ages are distributed across the industry.
age_category_counts = df['age_category'].value_counts().sort_index()
fig_age_cat = px.bar(
    x=age_category_counts.index.astype(str),
    y=age_category_counts.values,
    title='Number of Operating Plants by Age Category',
    labels={'x': 'Age Category', 'y': 'Number of Plants'},
    color=age_category_counts.values,
    color_continuous_scale='Greens'
)
fig_age_cat.update_layout(
    xaxis_title='Age Category',
    yaxis_title='Number of Plants',
    showlegend=False,
    height=400
)
fig_age_cat.show()

# Scatter plot comparing plant age and production capacity.
# Reveals potential trends between age and productivity (e.g., newer plants being larger or more efficient).
fig_age_capacity = px.scatter(
    df.dropna(subset=['Plant age (years)', 'Nominal crude steel capacity (ttpa)']),
    x='Plant age (years)',
    y='Nominal crude steel capacity (ttpa)',
    color='Region',
    size='Nominal crude stee



Plants with age data: 853 out of 868

Age Statistics:
  Mean: 39.65 years
  Median: 26.99 years
  Min: 0 years
  Max: 286 years
  Std Dev: 35.83 years

Age Distribution by Category:
age_category
0-10 years      108
11-20 years     172
21-30 years     186
31-40 years      84
41-50 years      45
51-100 years    199
100+ years       56
Name: count, dtype: int64


KEY INSIGHTS FROM AGE ANALYSIS
  • Oldest operating plant: 286 years old
  • Newest operating plant: 0 years old
  • Median age: 27.0 years
  • Most common age range: 51-100 years
  • Plants over 100 years old: 56


### Question 3: Geographic Distribution
**Task:** Analyze the geographic distribution of steel plants.
- Which countries/regions have the most steel plants?
- What is the distribution of plants by company?


In [ ]:
# Count the number of plants by country and by region to understand global distribution.
plants_by_country = df['Country/Area'].value_counts()
plants_by_region = df['Region'].value_counts()

# Display the top 15 countries with the highest number of operating steel plants.
print("Top 15 Countries by Number of Operating Steel Plants:\n")
print(plants_by_country.head(15))
print(f"\nTotal countries: {len(plants_by_country)}")

# Display total plant counts grouped by region.
print("Plants by Region:\n")
print(plants_by_region.sort_values(ascending=False))

# --- Visualization 1: Top 15 countries ---
# Create a horizontal bar chart showing which countries have the largest number of operating plants.
# This highlights national-level concentration within the steel industry.
top_15_countries = plants_by_country.head(15)
fig1 = px.bar(
    x=top_15_countries.values,
    y=top_15_countries.index,
    orientation='h',
    title='Top 15 Countries by Number of Operating Steel Plants',
    labels={'x': 'Number of Plants', 'y': 'Country'},
    color=top_15_countries.values,
    color_continuous_scale='Blues'
)
fig1.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    showlegend=False,
    height=500
)
fig1.show()

# --- Visualization 2: Regional distribution ---
# Generate a pie chart summarizing plant counts by region.
# The chart shows how the global steel industry is distributed across continents or regions.
fig2 = px.pie(
    values=plants_by_region.values,
    names=plants_by_region.index,
    title='Regional Distribution of Operating Steel Plants',
    hole=0.3  # Creates a donut-style chart for cleaner presentation
)
fig2.update_traces(textposition='inside', textinfo='percent+label')
fig2.update_layout(height=500)
fig2.show()

# --- Key insights summary ---
# Summarize major takeaways from the country and regional data analysis.
print(f"\nKey Insights:")
print(f"  • China leads with {plants_by_country.iloc[0]} plants ({plants_by_country.iloc[0]/len(df)*100:.1f}% of total)")
print(f"  • Top 5 countries account for {plants_by_country.head(5).sum()} plants ({plants_by_country.head(5).sum()/len(df)*100:.1f}% of total)")
print(f"  • {plants_by_region.idxmax()} has the most plants with {plants_by_region.max()}")


Top 15 Countries by Number of Operating Steel Plants:

Country/Area
China            289
India             74
United States     70
Japan             38
Iran              31
Russia            27
Türkiye           25
Brazil            22
Italy             22
Vietnam           20
South Korea       15
Spain             14
Germany           13
Mexico            13
Thailand          11
Name: count, dtype: int64

Total countries: 80
Plants by Region:

Region
Asia Pacific               497
Europe                     132
North America               91
Middle East                 51
Central & South America     37
Africa                      30
Eurasia                     30
Name: count, dtype: int64



Key Insights:
  • China leads with 289 plants (33.3% of total)
  • Top 5 countries account for 502 plants (57.8% of total)
  • Asia Pacific has the most plants with 497


In [ ]:
# Count the number of operating plants owned by each company.
# This helps identify which companies have the largest presence in steel production.
plants_by_owner = df['Owner'].value_counts()

# Display the top 20 companies ranked by number of plants.
print("Top 20 Companies (Owner) by Number of Operating Steel Plants:\n")
print(plants_by_owner.head(20))

# --- Summary statistics for company ownership distribution ---
# Provide general insights into how plant ownership is distributed across companies.
print("\n" + "="*60 + "\n")
print("Company Statistics (by Owner):")
print(f"  Total unique companies: {len(plants_by_owner)}")  # Total distinct owners
print(f"  Mean plants per company: {plants_by_owner.mean():.2f}")  # Average number of plants per company
print(f"  Median plants per company: {plants_by_owner.median():.1f}")  # Typical (median) ownership count
print(f"  Companies with only 1 plant: {(plants_by_owner == 1).sum()} "
      f"({(plants_by_owner == 1).sum()/len(plants_by_owner)*100:.1f}%)")  # Share of single-plant owners
print(f"  Companies with 5+ plants: {(plants_by_owner >= 5).sum()}")  # Number of large multi-plant companies

# --- Visualization: Top 20 companies by plant ownership ---
# Horizontal bar chart for better readability of company names.
# Shows which corporations dominate the steel industry by number of operating sites.
top_20_companies = plants_by_owner.head(20)
fig = px.bar(
    x=top_20_companies.values,
    y=top_20_companies.index,
    orientation='h',
    title='Top 20 Companies by Number of Operating Plants (Owner)',
    labels={'x': 'Number of Plants', 'y': 'Company'},
    color=top_20_companies.values,
    color_continuous_scale='Greens'
)
fig.update_layout(
    yaxis={'categoryorder': 'total ascending'},  # Sort bars by total
    showlegend=False,
    height=600
)
fig.show()

# --- Key insight ---
# Summarize concentration of ownership among the largest players.
print(f"\nTop 5 companies account for {plants_by_owner.head(5).sum()} plants "
      f"({plants_by_owner.head(5).sum()/len(df)*100:.1f}% of total)")


Top 20 Companies (Owner) by Number of Operating Steel Plants:

Owner
Nucor Corp                          11
Cleveland-Cliffs Inc                10
Nippon Steel Corp                    8
Gerdau Ameristeel Corp               7
Commercial Metals Co                 7
Steel Authority of India Ltd         6
ArcelorMittal Brasil SA              6
Steel Dynamics Inc                   6
Rungta Mines Ltd                     5
ArcelorMittal SA                     4
JSW Steel Ltd                        4
Mobarakeh Steel Co                   4
JFE Steel Corp                       4
Tokyo Steel Manufacturing Co Ltd     4
Government of North Korea            4
Angang Steel Co Ltd                  3
Gerdau Acos Longos SA                3
Acciaierie Venete SpA                3
Tata Steel Ltd                       3
Godo Steel Ltd                       3
Name: count, dtype: int64


Company Statistics (by Owner):
  Total unique companies: 731
  Mean plants per company: 1.19
  Median plants per company: 1


Top 5 companies account for 43 plants (5.0% of total)


### Question 4: Capacity Analysis
**Task:** Analyze the capacity distribution.
- What is the total global steel production capacity?
- Which companies have the highest total capacity?
- How does capacity vary by region?


In [ ]:
# --- Global capacity overview ---
# Calculate total global steel production capacity and number of plants with valid capacity data.
total_capacity = df['Nominal crude steel capacity (ttpa)'].sum()
plants_with_capacity = df['Nominal crude steel capacity (ttpa)'].notna().sum()

# Display overall capacity metrics for the dataset.
print("Global Steel Production Capacity:\n")
print(f"  Total capacity: {total_capacity:,.0f} ttpa")  # Total steel output capacity across all plants
print(f"  Plants with capacity data: {plants_with_capacity} out of {len(df)}")  # Data completeness
print(f"  Average capacity per plant: {df['Nominal crude steel capacity (ttpa)'].mean():,.0f} ttpa")  # Mean plant size

# --- Regional analysis ---
# Group data by region to compare total, count, and average plant capacity.
capacity_by_region = (
    df.groupby('Region')['Nominal crude steel capacity (ttpa)']
    .agg(['sum', 'count', 'mean'])
    .round(0)
)
capacity_by_region.columns = ['Total Capacity (ttpa)', 'Number of Plants', 'Average Capacity (ttpa)']
capacity_by_region = capacity_by_region.sort_values('Total Capacity (ttpa)', ascending=False)

# Display aggregated capacity data per region.
print("\nCapacity by Region:\n")
print(capacity_by_region)

# --- Visualization 1: Regional total capacity ---
# Horizontal bar chart showing total steel capacity per region.
# Useful for identifying which regions dominate global production.
fig1 = px.bar(
    capacity_by_region.reset_index(),
    x='Total Capacity (ttpa)',
    y='Region',
    orientation='h',
    title='Total Steel Production Capacity by Region',
    labels={'Total Capacity (ttpa)': 'Total Capacity (ttpa)', 'Region': 'Region'},
    color='Total Capacity (ttpa)',
    color_continuous_scale='Reds',
    text='Total Capacity (ttpa)'
)
fig1.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    showlegend=False,
    height=400
)
fig1.update_traces(
    texttemplate='%{text:,.0f}',  # Display formatted capacity values
    textposition='outside'
)
fig1.show()

# --- Visualization 2: Regional share of global capacity ---
# Pie (donut) chart highlighting each region’s share of total global capacity.
fig2 = px.pie(
    capacity_by_region.reset_index(),
    values='Total Capacity (ttpa)',
    names='Region',
    title='Global Steel Capacity Distribution by Region',
    hole=0.3
)
fig2.update_traces(textposition='inside', textinfo='percent+label')
fig2.update_layout(height=500)
fig2.show()

# --- Key takeaways ---
# Summarize the most important insights derived from capacity distribution.
print(f"\nKey Insights:")
print(f"  • {capacity_by_region.index[0]} has the highest capacity with {capacity_by_region.iloc[0, 0]:,.0f} ttpa "
      f"({capacity_by_region.iloc[0, 0]/total_capacity*100:.1f}%)")
print(f"  • Top 3 regions account for {capacity_by_region.head(3)['Total Capacity (ttpa)'].sum():,.0f} ttpa "
      f"({capacity_by_region.head(3)['Total Capacity (ttpa)'].sum()/total_capacity*100:.1f}%)")
print(f"  • Highest average plant capacity: {capacity_by_region['Average Capacity (ttpa)'].idxmax()} "
      f"({capacity_by_region['Average Capacity (ttpa)'].max():,.0f} ttpa)")


Global Steel Production Capacity:

  Total capacity: 2,004,617 ttpa
  Plants with capacity data: 815 out of 868
  Average capacity per plant: 2,460 ttpa

Capacity by Region:

                         Total Capacity (ttpa)  Number of Plants  \
Region                                                             
Asia Pacific                         1382183.0               464   
Europe                                208118.0               132   
North America                         148741.0                88   
Eurasia                                92565.0                29   
Middle East                            74973.0                43   
Central & South America                55713.0                31   
Africa                                 42324.0                28   

                         Average Capacity (ttpa)  
Region                                            
Asia Pacific                              2979.0  
Europe                                    1577.0  
North Am


Key Insights:
  • Asia Pacific has the highest capacity with 1,382,183 ttpa (68.9%)
  • Top 3 regions account for 1,739,042 ttpa (86.8%)
  • Highest average plant capacity: Eurasia (3,192 ttpa)


In [ ]:
# --- Calculate total production capacity and plant count per company (Owner) ---
capacity_by_owner = df.groupby('Owner')['Nominal crude steel capacity (ttpa)'].agg(['sum', 'count']).round(0)
capacity_by_owner.columns = ['Total Capacity (ttpa)', 'Number of Plants']
capacity_by_owner = capacity_by_owner.sort_values('Total Capacity (ttpa)', ascending=False)

# --- Display top companies by production capacity ---
print("Top 20 Companies by Total Steel Production Capacity:\n")
print(capacity_by_owner.head(20))

print("\n" + "="*60)
print("\nCapacity Concentration Statistics:")

# --- Quantify market concentration among top producers ---
print(f"  Top 5 companies control: {capacity_by_owner.head(5)['Total Capacity (ttpa)'].sum():,.0f} ttpa ({capacity_by_owner.head(5)['Total Capacity (ttpa)'].sum()/total_capacity*100:.1f}%)")
print(f"  Top 10 companies control: {capacity_by_owner.head(10)['Total Capacity (ttpa)'].sum():,.0f} ttpa ({capacity_by_owner.head(10)['Total Capacity (ttpa)'].sum()/total_capacity*100:.1f}%)")
print(f"  Top 20 companies control: {capacity_by_owner.head(20)['Total Capacity (ttpa)'].sum():,.0f} ttpa ({capacity_by_owner.head(20)['Total Capacity (ttpa)'].sum()/total_capacity*100:.1f}%)")

# --- Visualize top 20 companies by total production capacity ---
top_20_capacity = capacity_by_owner.head(20).reset_index()
fig1 = px.bar(
    top_20_capacity,
    x='Total Capacity (ttpa)',
    y='Owner',
    orientation='h',
    title='Top 20 Companies by Total Steel Production Capacity',
    labels={'Total Capacity (ttpa)': 'Total Capacity (ttpa)', 'Owner': 'Company'},
    color='Total Capacity (ttpa)',
    color_continuous_scale='Oranges',
    hover_data=['Number of Plants']
)
fig1.update_layout(yaxis={'categoryorder': 'total ascending'}, showlegend=False, height=700)
fig1.show()

# --- Summarize total capacity by region ---
region_capacity = df.groupby('Region')['Nominal crude steel capacity (ttpa)'].sum().reset_index()
region_capacity.columns = ['Region', 'Total Capacity (ttpa)']

# --- Identify top 5 producing countries per region ---
top_countries_by_region = df.groupby(['Region', 'Country/Area'])['Nominal crude steel capacity (ttpa)'].sum().reset_index()
top_countries_by_region = top_countries_by_region.sort_values('Nominal crude steel capacity (ttpa)', ascending=False).groupby('Region').head(5)

# --- Visualize regional breakdown with top countries in each region ---
fig2 = px.bar(
    top_countries_by_region,
    x='Nominal crude steel capacity (ttpa)',
    y='Region',
    color='Country/Area',
    orientation='h',
    title='Steel Production Capacity by Region and Top Countries',
    labels={'Nominal crude steel capacity (ttpa)': 'Capacity (ttpa)', 'Region': 'Region'},
    barmode='stack'
)
fig2.update_layout(height=500)
fig2.show()

# --- Print out the top country in each region based on capacity ---
print(f"\nRegional Capacity Leaders:")
for region in capacity_by_region.index:
    top_country = df[df['Region'] == region].groupby('Country/Area')['Nominal crude steel capacity (ttpa)'].sum().idxmax()
    top_capacity = df[df['Region'] == region].groupby('Country/Area')['Nominal crude steel capacity (ttpa)'].sum().max()
    print(f"  • {region}: {top_country} ({top_capacity:,.0f} ttpa)")


Top 20 Companies by Total Steel Production Capacity:

                                                Total Capacity (ttpa)  \
Owner                                                                   
POSCO Holdings Inc                                            40700.0   
Nippon Steel Corp                                             35395.0   
Angang Steel Co Ltd                                           30250.0   
JSW Steel Ltd                                                 28359.0   
Cleveland-Cliffs Inc                                          26377.0   
Hyundai Steel Co                                              24297.0   
JFE Steel Corp                                                20469.0   
Steel Authority of India Ltd                                  20132.0   
Baoshan Iron & Steel Co Ltd                                   19800.0   
Tata Steel Ltd                                                19720.0   
Nucor Corp                                                    17737.0 


Regional Capacity Leaders:
  • Asia Pacific: China (955,240 ttpa)
  • Europe: Türkiye (55,533 ttpa)
  • North America: United States (112,850 ttpa)
  • Eurasia: Russia (84,665 ttpa)
  • Middle East: Iran (43,899 ttpa)
  • Central & South America: Brazil (43,602 ttpa)
  • Africa: Egypt (16,600 ttpa)


---
## Part 3: Geospatial Visualization with Plotly (15 minutes)

Create interactive maps to visualize the steel plants' locations and characteristics.


### Exercise 1: Basic Scatter Map
**Task:** Create a scatter map showing all steel plant locations.
- Use latitude and longitude for positioning
- Color points by country or region
- Add hover information showing plant name, company, and capacity


In [ ]:
# --- Create a world map to visualize the geographic distribution of steel plants ---
fig = px.scatter_geo(
    df,
    lat='latitude',
    lon='longitude',
    color='Region',  # Different colors represent regions for better visual grouping
    hover_name='Plant name (English)',  # Display plant name when hovering
    hover_data={
        'Owner': True,  # Show company name
        'Nominal crude steel capacity (ttpa)': ':.0f',  # Format capacity with no decimals
        'Country/Area': True,  # Show the country
        'latitude': False,  # Hide raw coordinates from hover
        'longitude': False
    },
    title='Global Distribution of Operating Steel Plants',
    projection='natural earth'  # Use a natural earth-style world projection
)

# --- Customize the map appearance (land, ocean, borders, etc.) ---
fig.update_layout(
    height=600,
    geo=dict(
        showland=True,
        landcolor='rgb(243, 243, 243)',  # Light gray for land areas
        coastlinecolor='rgb(204, 204, 204)',  # Subtle coastline outline
        showocean=True,
        oceancolor='rgb(230, 245, 255)',  # Light blue for oceans
        showcountries=True,
        countrycolor='rgb(204, 204, 204)'  # Soft gray borders between countries
    )
)

# --- Display the interactive world map ---
fig.show()


### Exercise 2: Sized Markers by Capacity
**Task:** Create a map where marker size represents plant capacity.
- Larger markers for higher capacity plants
- Color by company
- Include interactive hover details


In [ ]:
# --- Filter dataset to include only plants with valid capacity data ---
# This ensures that only entries with available production capacity are visualized.
df_with_capacity = df.dropna(subset=['Nominal crude steel capacity (ttpa)'])

# --- Create a geographic scatter plot of steel plants ---
# Bubble size represents plant capacity, and color differentiates companies.
fig = px.scatter_geo(
    df_with_capacity,
    lat='latitude',
    lon='longitude',
    size='Nominal crude steel capacity (ttpa)',  # Scale point size by plant capacity
    color='Owner',  # Color points by company ownership
    hover_name='Plant name (English)',  # Show plant name when hovering
    hover_data={
        'Owner': True,
        'Nominal crude steel capacity (ttpa)': ':.0f',  # Format numbers neatly
        'Country/Area': True,
        'latitude': False,
        'longitude': False
    },
    title='Steel Plants Sized by Capacity and Colored by Company',
    projection='natural earth',  # Use a natural-looking global projection
    size_max=20  # Limit maximum bubble size for better readability
)

# --- Customize layout and map style ---
# Enhances clarity and visual appeal while maintaining geographic context.
fig.update_layout(
    height=600,
    showlegend=False,
    geo=dict(
        showland=True,
        landcolor='rgb(243, 243, 243)',  # Light gray for land
        coastlinecolor='rgb(204, 204, 204)',
        showocean=True,
        oceancolor='rgb(230, 245, 255)',  # Soft blue for ocean areas
        showcountries=True,
        countrycolor='rgb(204, 204, 204)'  # Subtle gray borders
    )
)

# --- Display the map ---
fig.show()

### Exercise 3: Density Heatmap
**Task:** Create a density map showing concentration of steel plants.
- Use Plotly's density_mapbox to show clustering
- Identify regions with high plant density


In [ ]:
# --- Create a density heatmap of steel plant locations ---
# This visualization highlights geographic regions with high concentrations of plants.
fig = px.density_mapbox(
    df,
    lat='latitude',
    lon='longitude',
    radius=10,  # Size of influence area for each plant
    zoom=1,  # Initial zoom level for global view
    center={'lat': 30, 'lon': 50},  # Center the map roughly over major industrial regions
    mapbox_style='open-street-map',  # Use open-source map style
    title='Density Heatmap of Steel Plant Locations',
    color_continuous_scale='Hot'  # Color intensity represents density (hot = more plants)
)

# Set overall figure height for better visibility
fig.update_layout(height=600)

# Display the interactive heatmap
fig.show()

# --- Key regions with high steel plant density ---
# Provide quick insights for interpretation of the heatmap.
print("\nRegions with High Plant Density:")
print("  • Eastern China (Yangtze River Delta, Pearl River Delta)")
print("  • Northern India")
print("  • Eastern United States")
print("  • Western Europe")
print("  • Japan (coastal regions)")

C:\Users\walid\AppData\Local\Temp\ipykernel_35084\1487235573.py:1: DeprecationWarning:

*density_mapbox* is deprecated! Use *density_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/




Regions with High Plant Density:
  • Eastern China (Yangtze River Delta, Pearl River Delta)
  • Northern India
  • Eastern United States
  • Western Europe
  • Japan (coastal regions)


---
## Part 4: Merging Environmental Data with Assets

Integrate environmental data (e.g., air quality, emissions, proximity to water sources) with steel plant locations.


### Exercise 1: Load Environmental Data
**Task:** Load the environmental dataset and inspect it.

- [Litpop database](https://www.research-collection.ethz.ch/entities/researchdata/12dcfc4f-9d03-463a-8d6b-76c0dc73cdc8)

- Expected columns: location_id, latitude, longitude, population density, activity etc.


In [ ]:
# Load environmental data



In [ ]:
# Inspect environmental data



### Exercise 2: Spatial Join or Nearest Neighbor Matching
**Task:** Merge environmental data with steel plants based on geographic proximity.
- Use nearest neighbor matching or spatial join
- Consider using geopandas for distance calculations
- Match each plant to the nearest environmental monitoring station


In [ ]:
# Calculate distances or perform spatial join
# Hint: You might calculate haversine distance or use a spatial library



In [ ]:
# Merge datasets



### Exercise 3: Visualize Merged Data
**Task:** Create a map showing steel plants colored by environmental metrics.
- Color plants by air quality index or other environmental indicators
- Size by capacity
- Add hover details with both plant and environmental information


In [ ]:
# Create visualization of merged data



---
## Part 5: Company-Level Aggregation

Aggregate data at the company level to analyze corporate footprints.


### PS: Some parts of parts 5 strongly rely on part 4's environmental metrics data set, thus we will have to "neglect" anything that has to do with environmental data in this part

### Exercise 1: Aggregate Metrics by Company
**Task:** Group plants by company and calculate aggregate metrics.
- Total capacity per company
- Number of plants per company
- Average environmental metrics per company
- Geographic spread (e.g., number of countries)


In [ ]:
# Group by company and aggregate
# This cell computes total capacity, number of plants, average capacity,
# number of unique countries and regions, and basic plant-age stats when available.
import warnings
warnings.filterwarnings('ignore')

# Ensure key columns exist
for col in ['Owner', 'Nominal crude steel capacity (ttpa)', 'Country/Area', 'Region', 'Plant age (years)', 'latitude', 'longitude', 'Plant name (English)']:
    if col not in df.columns:
        df[col] = pd.NA

# Convert numeric columns
df['Nominal crude steel capacity (ttpa)'] = pd.to_numeric(df['Nominal crude steel capacity (ttpa)'], errors='coerce')
df['Plant age (years)'] = pd.to_numeric(df['Plant age (years)'], errors='coerce')
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

# Aggregations by Owner (company)
agg_funcs = {
    'Nominal crude steel capacity (ttpa)': ['sum', 'mean', 'median', 'count'],
    'Plant age (years)': ['mean', 'median', 'count'],
    'Country/Area': lambda x: x.nunique(),
    'Region': lambda x: x.nunique(),
}
company_agg = df.groupby('Owner').agg(agg_funcs)
# Flatten MultiIndex columns
company_agg.columns = ['_'.join(filter(None, map(str, col))).strip() for col in company_agg.columns.values]
company_agg = company_agg.rename(columns={
    'Nominal crude steel capacity (ttpa)_sum': 'total_capacity_tppa',
    'Nominal crude steel capacity (ttpa)_mean': 'avg_capacity_tppa',
    'Nominal crude steel capacity (ttpa)_median': 'median_capacity_tppa',
    'Nominal crude steel capacity (ttpa)_count': 'plants_with_capacity_count',
    'Plant age (years)_mean': 'avg_plant_age_years',
    'Plant age (years)_median': 'median_plant_age_years',
    'Plant age (years)_count': 'plants_with_age_count',
    'Country/Area_<lambda>': 'num_countries',
    'Region_<lambda>': 'num_regions'
})

# Number of plants per company (including those without capacity)
company_agg['num_plants'] = df.groupby('Owner').size()

# Replace NaN total_capacity with 0 for companies with no capacity data
company_agg['total_capacity_tppa'] = company_agg['total_capacity_tppa'].fillna(0)

# Sort by total capacity
company_agg = company_agg.sort_values('total_capacity_tppa', ascending=False)

# Quick display of top companies
display(company_agg.head(10))

# Keep for later cells
capacity_by_company = company_agg.copy()



,total_capacity_tppa,avg_capacity_tppa,median_capacity_tppa,plants_with_capacity_count,avg_plant_age_years,median_plant_age_years,plants_with_age_count,num_countries,num_regions,num_plants
Owner,,,,,,,,,,
POSCO Holdings Inc,40700.0,20350.000000,20350.0,2,45.000000,45.0,2,1,1,2
Nippon Steel Corp,35395.0,5056.428571,3432.0,7,80.535000,73.5,8,1,1,8
Angang Steel Co Ltd,30250.0,10083.333333,7500.0,3,21.380000,19.0,3,1,1,3
JSW Steel Ltd,28359.0,7089.750000,7600.0,4,27.250000,29.0,4,1,1,4
Cleveland-Cliffs Inc,26377.0,2930.777778,2722.0,9,100.900000,108.5,10,2,1,10
Hyundai Steel Co,24297.0,8099.000000,4697.0,3,45.500000,45.5,2,1,1,3
JFE Steel Corp,20469.0,5117.250000,3307.5,4,60.840000,59.0,4,1,1,4
Steel Authority of India Ltd,20132.0,3355.333333,2890.5,6,69.781667,66.0,6,1,1,6
Baoshan Iron & Steel Co Ltd,19800.0,19800.000000,19800.0,1,48.000000,48.0,1,1,1,1


### Exercise 2: Company Headquarters or Centroid
**Task:** Calculate a representative location for each company.
- Option 1: Use the centroid of all plant locations
- Option 2: Use the location of the largest plant
- Option 3: Assign actual headquarters coordinates


In [ ]:
# Calculate company representative locations
# We'll compute two options: centroid of plant coordinates and the coords of the largest plant.
# Use centroid by default; if coordinates missing for a company, fall back to largest-plant coords or leave NaN.

# Prepare a dataframe of plants with coordinates
plants_coords = df.dropna(subset=['latitude', 'longitude']).copy()

# Centroid per company (mean latitude/longitude)
centroid = plants_coords.groupby('Owner')[['latitude', 'longitude']].mean().rename(columns={
    'latitude': 'centroid_lat',
    'longitude': 'centroid_lon'
})

# Largest plant coords per company (by capacity)
plants_with_cap = df.dropna(subset=['Nominal crude steel capacity (ttpa)', 'latitude', 'longitude']).copy()
largest_idx = plants_with_cap.groupby('Owner')['Nominal crude steel capacity (ttpa)'].idxmax()
largest = plants_with_cap.loc[largest_idx, ['Owner', 'latitude', 'longitude', 'Plant name (English)', 'Nominal crude steel capacity (ttpa)']].set_index('Owner')
largest = largest.rename(columns={'latitude': 'largest_lat', 'longitude': 'largest_lon', 'Plant name (English)': 'largest_plant_name', 'Nominal crude steel capacity (ttpa)': 'largest_plant_capacity_tppa'})

# Combine into company_agg (which should exist from previous step)
company_locations = capacity_by_company.copy()
company_locations = company_locations.join(centroid, how='left')
company_locations = company_locations.join(largest[['largest_lat', 'largest_lon', 'largest_plant_name', 'largest_plant_capacity_tppa']], how='left')

# Choose representative location: prefer centroid if available, else largest plant coords
company_locations['rep_lat'] = company_locations['centroid_lat'].fillna(company_locations['largest_lat'])
company_locations['rep_lon'] = company_locations['centroid_lon'].fillna(company_locations['largest_lon'])

# Flag which method was used
def pick_method(row):
    if pd.notna(row['centroid_lat']) and pd.notna(row['centroid_lon']):
        return 'centroid'
    if pd.notna(row['largest_lat']) and pd.notna(row['largest_lon']):
        return 'largest_plant'
    return pd.NA

company_locations['rep_location_method'] = company_locations.apply(pick_method, axis=1)

# Display a sample
display(company_locations[['num_plants', 'total_capacity_tppa', 'num_countries', 'rep_lat', 'rep_lon', 'rep_location_method']].head(10))

# Keep for later cells
company_aggregations = company_locations.copy()


,num_plants,total_capacity_tppa,num_countries,rep_lat,rep_lon,rep_location_method
Owner,,,,,,
POSCO Holdings Inc,2,40700.0,1,35.464698,128.571579,centroid
Nippon Steel Corp,8,35395.0,1,35.607220,136.334452,centroid
Angang Steel Co Ltd,3,30250.0,1,40.995235,121.827942,centroid
JSW Steel Ltd,4,28359.0,1,16.862409,77.910008,centroid
Cleveland-Cliffs Inc,10,26377.0,2,41.265330,-82.565167,centroid
Hyundai Steel Co,3,24297.0,1,36.826980,127.573374,centroid
JFE Steel Corp,4,20469.0,1,35.703261,137.067622,centroid
Steel Authority of India Ltd,6,20132.0,1,22.968698,85.636231,centroid
Baoshan Iron & Steel Co Ltd,1,19800.0,1,31.416230,121.440087,centroid


### Exercise 3: Visualize Company-Level Data
**Task:** Create a map showing companies with aggregated metrics.
- Show one marker per company at the representative location
- Size by total capacity
- Color by average environmental impact
- Hover information with company summary statistics


### We do not have the Environmental impact data, so we'll color the dots using some other metric, being avg capacity in this case

In [ ]:
# Create company-level visualization
# Map: one point per company at representative location, sized by total capacity.
# Color metric can be selected from available company-level columns; defaults to average capacity.
import plotly.express as px

# Prepare dataframe for plotting - drop companies without rep coords
plot_df = company_aggregations.reset_index().rename(columns={'index': 'Owner'})
plot_df = plot_df.dropna(subset=['rep_lat', 'rep_lon']).copy()

# Cap size values for plotting to avoid huge markers; convert tppa to numeric
plot_df['total_capacity_tppa'] = pd.to_numeric(plot_df.get('total_capacity_tppa', 0), errors='coerce').fillna(0)
plot_df['size_for_plot'] = (plot_df['total_capacity_tppa'] / max(1, plot_df['total_capacity_tppa'].max())) * 40 + 5

# Choose a color metric (change this variable to another column if you prefer)
# Options we provide: 'avg_capacity_tppa', 'total_capacity_tppa', 'num_plants', 'avg_plant_age_years', 'num_countries'
preferred_color_metric = 'avg_capacity_tppa'

# Fallback order if preferred metric is missing
fallback_metrics = ['avg_capacity_tppa', 'total_capacity_tppa', 'num_plants', 'num_countries', 'avg_plant_age_years']
color_metric = preferred_color_metric if preferred_color_metric in plot_df.columns else None
if color_metric is None:
    for m in fallback_metrics:
        if m in plot_df.columns:
            color_metric = m
            break
if color_metric is None:
    # last-resort: create a simple metric (number of plants) if nothing else
    plot_df['num_plants'] = plot_df.get('num_plants', 1)
    color_metric = 'num_plants'

# Ensure color metric is numeric when appropriate
try:
    if plot_df[color_metric].dtype == object:
        plot_df[color_metric] = pd.to_numeric(plot_df[color_metric], errors='coerce')
except Exception:
    pass

# Build hover data dynamically and include the color metric
hover = {
    'total_capacity_tppa': ':.0f',
    'num_plants': True,
    'num_countries': True,
    'rep_location_method': True,
    'rep_lat': False,
    'rep_lon': False
}
if color_metric not in hover:
    hover[color_metric] = True

fig_map = px.scatter_geo(
    plot_df,
    lat='rep_lat',
    lon='rep_lon',
    size='size_for_plot',
    color=color_metric,
    hover_name='Owner',
    hover_data=hover,
    title=f'Company-Level Footprint: Representative Locations (color={color_metric})',
    projection='natural earth',
    color_continuous_scale='Viridis'
)
fig_map.update_layout(height=700, showlegend=False)
fig_map.show()

# Bar chart: Top 20 companies by total capacity
top20 = plot_df.sort_values('total_capacity_tppa', ascending=False).head(20)
fig_bar = px.bar(top20, x='total_capacity_tppa', y='Owner', orientation='h',
                 title='Top 20 Companies by Total Capacity (ttpa)',
                 labels={'total_capacity_tppa': 'Total capacity (ttpa)', 'Owner': 'Company'},
                 hover_data=['num_plants', 'num_countries'])
fig_bar.update_layout(height=700, yaxis={'categoryorder':'total ascending'})
fig_bar.show()



---
## Part 6: Streamlit Dashboard Integration

Prepare your visualizations for deployment in a Streamlit dashboard.


### Exercise 1: Create Dashboard Script Structure
**Task:** Create a Streamlit app file (`dashboard.py`) with the following structure:

```python
# Import streamlit and other necessary libraries

# Set page configuration

# Title and description

# Sidebar for filters
# - Company selector
# - Region/country filter
# - Capacity range slider

# Main content area
# - KPI metrics (total plants, total capacity, etc.)
# - Interactive map
# - Data table

# Footer with data sources and notes
```


### Exercise 1: Prepare Data for Dashboard
**Task:** Save your processed data to files that the dashboard will load.
- Export cleaned plant data
- Export merged environmental data
- Export company-level aggregations
- Save as CSV or Parquet for efficient loading


In [ ]:
import os
# Create output directory if it doesn't exist
output_dir = 'processed_data'
os.makedirs(output_dir, exist_ok=True)

# Saving the cleaned plant data
df.to_csv(f'{output_dir}/steel_plants_cleaned.csv', index=False)
print(f"Exported cleaned plant data: {len(df)} plants")

capacity_by_owner.to_csv(f'{output_dir}/capacity_by_company.csv')
print(f"Exported company capacity aggregations: {len(capacity_by_owner)} companies")

capacity_by_region.to_csv(f'{output_dir}/capacity_by_region.csv')
print(f"Exported regional capacity aggregations: {len(capacity_by_region)} regions")

plants_by_country_df = plants_by_country.to_frame('Number of Plants')
plants_by_country_df.to_csv(f'{output_dir}/plants_by_country.csv')
print(f"Exported country plant counts: {len(plants_by_country_df)} countries")

plants_by_owner_df = plants_by_owner.to_frame('Number of Plants')
plants_by_owner_df.to_csv(f'{output_dir}/plants_by_company.csv')
print(f"Exported company plant counts: {len(plants_by_owner_df)} companies")

print(f"\nAll files saved to '{output_dir}/' directory")

Exported cleaned plant data: 868 plants
Exported company capacity aggregations: 731 companies
Exported regional capacity aggregations: 7 regions
Exported country plant counts: 80 countries
Exported company plant counts: 731 companies

All files saved to 'processed_data/' directory


### Exercise 2: Display relevant information from your exploratory analysis into the dashboard

In [ ]:
# This cell is for notes/observations about your dashboard
# What works well?
# What could be improved?
# Any performance issues with large datasets?



Our Answer:

The dashboard works very well overall. The interactivity is smooth, and filtering plants by capacity, region, country, or company is straightforward and intuitive. The visualizations, including the global map, bar charts, and pie charts, effectively show the distribution and ranking of plants, making it suitable for presenting insights to non-technical clients in a consulting context.

There are some areas where the dashboard could be improved. For example, adding more visualizations to explore relationships between other plant features could provide deeper insights. Visualizing plant age against capacity, examining raw material sources like iron ore and met coal, or looking at parent company networks could add more depth. Additionally, allowing sorting or filtering by other fields such as plant age, SOE status, or raw material sources would make the dashboard more versatile. Including summary statistics per region or per company dynamically would also help, and if historical data were available, showing capacity growth over time would be valuable. On the map, clustering could improve readability in regions where plants are very dense.

Regarding performance, the dashboard handles the current dataset of about 1,200 plants very smoothly. However, if the dataset were to grow significantly, for example to 10,000 plants or more, some visualizations like scatter maps and bar charts might slow down. In such cases, using data aggregation, sampling, or WebGL-based plotting could help maintain performance.


---
## Lab Summary and Key Takeaways

**What you learned:**
- How to perform EDA on geospatial datasets
- Creating interactive maps with Plotly for geospatial data
- Merging spatial datasets based on geographic proximity
- Aggregating geospatial data at different levels (asset vs. company)
- Building interactive dashboards with Streamlit

**Next Steps:**
- Explore other geospatial libraries (GeoPandas, Folium, Kepler.gl)
- Learn about coordinate reference systems (CRS) and projections
- Practice with other datasets (buildings, utilities, transportation)
- Deploy your dashboard to Streamlit Cloud or other hosting services
